In [1]:
import json
import time
import requests
import json
import os
import pandas as pd
import warnings
import  arrow
from sqlalchemy import create_engine
import sys
from sqlalchemy import text
if sys.version_info.major<3:
    reload(sys)
    sys.setdefaultencoding("utf-8")
warnings.filterwarnings("ignore")

def update_china_data(unit=3600 * 2):
    # 中国各省数据
    baseUrl = "https://lab.ahusmart.com/nCoV/api/"
    url = baseUrl + 'area?latest=1'
    data = json.loads(requests.get(url).text)
    p_data = {}
    for r in data['results']:
        # 去除别的国家数据
        if r.get("provinceShortName") == None or r.get("country") != "中国":
            pass
        else:
            p_data[r["provinceShortName"]] = r["confirmedCount"]
    # 先对字典进行排序,按照value从大到小
    p_data = sorted(p_data.items(), key=lambda x: x[1], reverse=True)
    return p_data
def update_province_data(province):
    baseUrl = "https://lab.ahusmart.com/nCoV/api/"
    url = baseUrl + 'area?latest=1&province=' + province
    province_data = json.loads(requests.get(url).text)
    province_data['time'] = time.strftime(
        "%m-%d %H:%M", time.localtime(time.time()))
    # 取最新一条统计数据
    latest_data = province_data["results"][0]
    cities_data = []
    for c in latest_data["cities"]:
        city_data = []
        city_data.append(province)
        # 先在修正表中找
        '''
        if province_city_fixed[province][c["cityName"]]!=None:
            city_data.append(province_city_fixed[province][c["cityName"]])
        # 没有则延用原始名称
        else:
            city_data.append(c["cityName"])'
        '''
        try:
            city_data.append(province_city_fixed[province][c["cityName"]])
        except:
            city_data.append(c["cityName"])
        else:
            pass
        city_data.append(c["confirmedCount"])
        cities_data.append(city_data)
    return cities_data

In [4]:
baseUrl = "https://lab.ahusmart.com/nCoV/api/"
url = baseUrl + 'area?latest=1&province=' + '湖北省'
requests.get(url).text

'{"results":[{"country":"中国","provinceName":"湖北省","provinceShortName":"湖北","confirmedCount":7153,"suspectedCount":0,"curedCount":168,"deadCount":249,"cities":[{"cityName":"武汉","confirmedCount":3215,"curedCount":106,"deadCount":192,"locationId":420100,"suspectedCount":0},{"cityName":"黄冈","confirmedCount":726,"curedCount":17,"deadCount":14,"locationId":421100,"suspectedCount":0},{"cityName":"孝感","confirmedCount":628,"curedCount":2,"deadCount":12,"locationId":420900,"suspectedCount":0},{"cityName":"襄阳","confirmedCount":347,"curedCount":0,"deadCount":0,"locationId":420600,"suspectedCount":0},{"cityName":"随州","confirmedCount":304,"curedCount":0,"deadCount":1,"locationId":421300,"suspectedCount":0},{"cityName":"荆州","confirmedCount":287,"curedCount":2,"deadCount":4,"locationId":421000,"suspectedCount":0},{"cityName":"宜昌","confirmedCount":276,"curedCount":0,"deadCount":1,"locationId":420500,"suspectedCount":0},{"cityName":"荆门","confirmedCount":251,"curedCount":1,"deadCount":5,"locationId":4208

In [5]:
requests.get(url)

<Response [200]>

In [17]:
res = json.loads(requests.get(url).text)
print(type(res))

<class 'dict'>


In [27]:
for key,value in res.items():
    print("键:" , key)
    print("--------" )
    print("值:" , value)

键: results
--------
值: [{'country': '中国', 'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 7153, 'suspectedCount': 0, 'curedCount': 168, 'deadCount': 249, 'cities': [{'cityName': '武汉', 'confirmedCount': 3215, 'curedCount': 106, 'deadCount': 192, 'locationId': 420100, 'suspectedCount': 0}, {'cityName': '黄冈', 'confirmedCount': 726, 'curedCount': 17, 'deadCount': 14, 'locationId': 421100, 'suspectedCount': 0}, {'cityName': '孝感', 'confirmedCount': 628, 'curedCount': 2, 'deadCount': 12, 'locationId': 420900, 'suspectedCount': 0}, {'cityName': '襄阳', 'confirmedCount': 347, 'curedCount': 0, 'deadCount': 0, 'locationId': 420600, 'suspectedCount': 0}, {'cityName': '随州', 'confirmedCount': 304, 'curedCount': 0, 'deadCount': 1, 'locationId': 421300, 'suspectedCount': 0}, {'cityName': '荆州', 'confirmedCount': 287, 'curedCount': 2, 'deadCount': 4, 'locationId': 421000, 'suspectedCount': 0}, {'cityName': '宜昌', 'confirmedCount': 276, 'curedCount': 0, 'deadCount': 1, 'locationId': 4205

In [33]:
print(res['results'])


[{'country': '中国', 'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 7153, 'suspectedCount': 0, 'curedCount': 168, 'deadCount': 249, 'cities': [{'cityName': '武汉', 'confirmedCount': 3215, 'curedCount': 106, 'deadCount': 192, 'locationId': 420100, 'suspectedCount': 0}, {'cityName': '黄冈', 'confirmedCount': 726, 'curedCount': 17, 'deadCount': 14, 'locationId': 421100, 'suspectedCount': 0}, {'cityName': '孝感', 'confirmedCount': 628, 'curedCount': 2, 'deadCount': 12, 'locationId': 420900, 'suspectedCount': 0}, {'cityName': '襄阳', 'confirmedCount': 347, 'curedCount': 0, 'deadCount': 0, 'locationId': 420600, 'suspectedCount': 0}, {'cityName': '随州', 'confirmedCount': 304, 'curedCount': 0, 'deadCount': 1, 'locationId': 421300, 'suspectedCount': 0}, {'cityName': '荆州', 'confirmedCount': 287, 'curedCount': 2, 'deadCount': 4, 'locationId': 421000, 'suspectedCount': 0}, {'cityName': '宜昌', 'confirmedCount': 276, 'curedCount': 0, 'deadCount': 1, 'locationId': 420500, 'suspectedCount': 0

In [34]:
print(type(res['results']))

<class 'list'>


In [38]:
len(res['results'])

1

In [40]:
type(res['results'][0])

dict

In [41]:
print(res['results'][0])

{'country': '中国', 'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 7153, 'suspectedCount': 0, 'curedCount': 168, 'deadCount': 249, 'cities': [{'cityName': '武汉', 'confirmedCount': 3215, 'curedCount': 106, 'deadCount': 192, 'locationId': 420100, 'suspectedCount': 0}, {'cityName': '黄冈', 'confirmedCount': 726, 'curedCount': 17, 'deadCount': 14, 'locationId': 421100, 'suspectedCount': 0}, {'cityName': '孝感', 'confirmedCount': 628, 'curedCount': 2, 'deadCount': 12, 'locationId': 420900, 'suspectedCount': 0}, {'cityName': '襄阳', 'confirmedCount': 347, 'curedCount': 0, 'deadCount': 0, 'locationId': 420600, 'suspectedCount': 0}, {'cityName': '随州', 'confirmedCount': 304, 'curedCount': 0, 'deadCount': 1, 'locationId': 421300, 'suspectedCount': 0}, {'cityName': '荆州', 'confirmedCount': 287, 'curedCount': 2, 'deadCount': 4, 'locationId': 421000, 'suspectedCount': 0}, {'cityName': '宜昌', 'confirmedCount': 276, 'curedCount': 0, 'deadCount': 1, 'locationId': 420500, 'suspectedCount': 0}

In [44]:
dd = res['results'][0]
for key,value in dd.items():
    print("键:" , key)
    #print("--------" )
    print("值:" , value)

键: country
值: 中国
键: provinceName
值: 湖北省
键: provinceShortName
值: 湖北
键: confirmedCount
值: 7153
键: suspectedCount
值: 0
键: curedCount
值: 168
键: deadCount
值: 249
键: cities
值: [{'cityName': '武汉', 'confirmedCount': 3215, 'curedCount': 106, 'deadCount': 192, 'locationId': 420100, 'suspectedCount': 0}, {'cityName': '黄冈', 'confirmedCount': 726, 'curedCount': 17, 'deadCount': 14, 'locationId': 421100, 'suspectedCount': 0}, {'cityName': '孝感', 'confirmedCount': 628, 'curedCount': 2, 'deadCount': 12, 'locationId': 420900, 'suspectedCount': 0}, {'cityName': '襄阳', 'confirmedCount': 347, 'curedCount': 0, 'deadCount': 0, 'locationId': 420600, 'suspectedCount': 0}, {'cityName': '随州', 'confirmedCount': 304, 'curedCount': 0, 'deadCount': 1, 'locationId': 421300, 'suspectedCount': 0}, {'cityName': '荆州', 'confirmedCount': 287, 'curedCount': 2, 'deadCount': 4, 'locationId': 421000, 'suspectedCount': 0}, {'cityName': '宜昌', 'confirmedCount': 276, 'curedCount': 0, 'deadCount': 1, 'locationId': 420500, 'suspected

In [49]:
info_cities = dd['cities']
print(type(info_cities))
print(len(info_cities))

<class 'list'>
17


In [50]:
info_cities[4]

{'cityName': '随州',
 'confirmedCount': 304,
 'curedCount': 0,
 'deadCount': 1,
 'locationId': 421300,
 'suspectedCount': 0}

In [51]:
info_cities[4]['cityName']

'随州'

In [52]:
info_cities[4]['confirmedCount']

304

In [53]:
pd.DataFrame(info_cities[4])

ValueError: If using all scalar values, you must pass an index

In [56]:
list(info_cities[4].keys())

['cityName',
 'confirmedCount',
 'curedCount',
 'deadCount',
 'locationId',
 'suspectedCount']

In [57]:
list(info_cities[4].values())

['随州', 304, 0, 1, 421300, 0]

In [62]:
DF=pd.DataFrame(list(info_cities[4].values())).T #,columns=list(info_cities[4].keys())
DF.columns =list(info_cities[4].keys())

In [63]:
DF

,cityName,confirmedCount,curedCount,deadCount,locationId,suspectedCount
0,随州,304,0,1,421300,0


In [65]:
for key,value in info_cities[4].items():
    print("键:" , key)
    #print("--------" )
    print("值:" , value)

键: cityName
值: 随州
键: confirmedCount
值: 304
键: curedCount
值: 0
键: deadCount
值: 1
键: locationId
值: 421300
键: suspectedCount
值: 0


In [91]:
df_empty =pd.DataFrame(columns=list(info_cities[4].keys()))#  空       T #,columns=list(info_cities[4].keys())
#df.columns =list(info_cities[4].keys())

In [92]:
df_empty

,cityName,confirmedCount,curedCount,deadCount,locationId,suspectedCount


In [93]:
for each_city in info_cities:
    DF=pd.DataFrame(list(each_city.values())).T #,columns=list(info_cities[4].keys())
    DF.columns =list(each_city.keys())
    df_empty = df_empty.append(DF,ignore_index=True)
    
    

In [94]:
df_empty.drop_duplicates()

,cityName,confirmedCount,curedCount,deadCount,locationId,suspectedCount
0,武汉,3215,106,192,420100,0
1,黄冈,726,17,14,421100,0
2,孝感,628,2,12,420900,0
3,襄阳,347,0,0,420600,0
4,随州,304,0,1,421300,0
5,荆州,287,2,4,421000,0
6,宜昌,276,0,1,420500,0
7,荆门,251,1,5,420800,0
8,鄂州,227,0,9,420700,0
9,黄石,209,1,2,420200,0


In [95]:
import json
import time
import requests
import os
import pandas as pd
baseUrl = "https://lab.ahusmart.com/nCoV/api/"
url = baseUrl + 'area?latest=1&province=' + '湖北省'
res = json.loads(requests.get(url).text)
print(type(res))
dd = res['results'][0]
print(res['results'][0])

info_cities = dd['cities']
print(type(info_cities))
print(len(info_cities))


#DF=pd.DataFrame(list(info_cities[4].values())).T #,columns=list(info_cities[4].keys())
#DF.columns =list(info_cities[4].keys())
df_empty =pd.DataFrame(columns=list(info_cities[4].keys()))#  空
for each_city in info_cities:
    DF = pd.DataFrame(list(each_city.values())).T  # ,columns=list(info_cities[4].keys())
    DF.columns = list(each_city.keys())
    df_empty = df_empty.append(DF, ignore_index=True)
df_final = df_empty.drop_duplicates()

print(df_final)



<class 'dict'>
{'country': '中国', 'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 7153, 'suspectedCount': 0, 'curedCount': 168, 'deadCount': 249, 'cities': [{'cityName': '武汉', 'confirmedCount': 3215, 'curedCount': 106, 'deadCount': 192, 'locationId': 420100, 'suspectedCount': 0}, {'cityName': '黄冈', 'confirmedCount': 726, 'curedCount': 17, 'deadCount': 14, 'locationId': 421100, 'suspectedCount': 0}, {'cityName': '孝感', 'confirmedCount': 628, 'curedCount': 2, 'deadCount': 12, 'locationId': 420900, 'suspectedCount': 0}, {'cityName': '襄阳', 'confirmedCount': 347, 'curedCount': 0, 'deadCount': 0, 'locationId': 420600, 'suspectedCount': 0}, {'cityName': '随州', 'confirmedCount': 304, 'curedCount': 0, 'deadCount': 1, 'locationId': 421300, 'suspectedCount': 0}, {'cityName': '荆州', 'confirmedCount': 287, 'curedCount': 2, 'deadCount': 4, 'locationId': 421000, 'suspectedCount': 0}, {'cityName': '宜昌', 'confirmedCount': 276, 'curedCount': 0, 'deadCount': 1, 'locationId': 420500, 'susp